<img align="Left" alt="Dados_ao_Cubo" src="../shared_content/DADOS-AO-CUBO-Versão-04-1.png" />
<img align="right" alt="MLNet" src="../shared_content/EJ3KsfJWsAEtDaZ.png" />

## Davi Ramos -> Cientista de Dados 👋
(davi.info@gmail.com)

[![Linkedin Badge](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=Linkedin&logoColor=white&link=https://www.linkedin.com/in/davi-ramos/)](https://www.linkedin.com/in/davi-ramos/)
[![Twitter Badge](https://img.shields.io/badge/-Twitter-1DA1F2?style=flat-square&logo=Twitter&logoColor=white&link=https://twitter.com/Daviinfo/)](https://twitter.com/Daviinfo/)
<a href="https://github.com/DaviRamos"><img src="https://img.shields.io/github/followers/DaviRamos.svg?label=GitHub&style=social" alt="GitHub"></a>

# ML.Net - Auto ML

## Este exemplo irá demontrar o Uso de AutoML com ML.Net

In [1]:
// Instalar os Pacotes do Nuget

// ML.NET
#r "nuget:Microsoft.ML"  

// ML.NET AutoML
#r "nuget:Microsoft.ML.AutoML" 

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.ML version 1.5.2

Installed package Microsoft.ML.AutoML version 0.17.2

In [2]:
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;

using static Microsoft.ML.TrainCatalogBase;
using static Microsoft.ML.DataOperationsCatalog;

In [3]:
// Caminho dos arquivos
string housingPath = "./Datasets/Housing/housing.csv";
string ModelPath = @"./HouseModelv3.zip";
private static uint ExperimentTime = 10;

In [4]:
public class ModelInput
{
    [LoadColumn(0)]
    public float Longitude { get; set; }
    [LoadColumn(1)]
    public float Latitude { get; set; }
    [LoadColumn(2)]
    public float Housing_median_age { get; set; }
    [LoadColumn(3)]
    public float Total_rooms { get; set; }
    [LoadColumn(4)]
    public float Total_bedrooms { get; set; }
    [LoadColumn(5)]
    public float Population { get; set; }
    [LoadColumn(6)]
    public float Households { get; set; }
    [LoadColumn(7)]
    public float Median_income { get; set; }
    [ColumnName("Label"), LoadColumn(8)]
    public float Median_house_value { get; set; }
    [LoadColumn(9)]
    public string Ocean_proximity { get; set; }
}

public class ModelOutput
{
    [ColumnName("Score")]
    public float Score { get; set; }
}

In [31]:
private static void PrintMetrics(RegressionMetrics metrics)
{
    Console.WriteLine($"MeanAbsoluteError: {metrics.MeanAbsoluteError:0.##}");
    Console.WriteLine($"MeanSquaredError: {metrics.MeanSquaredError:0.##}");
    Console.WriteLine($"RootMeanSquaredError: {metrics.RootMeanSquaredError:0.##}");
    Console.WriteLine($"RSquared: {metrics.RSquared:0.##}");
}

In [6]:
// Criar o MLContext para ser compartilhado entre os objetos de fluxo de trabalho de criação do modelo
// Definir uma semente aleatória para resultados repetíveis/determinísticos nos vários treinamentos.
MLContext mlContext = new MLContext(seed: 1);

In [ ]:
private static IDataView trainDataView;
private static IDataView testDataView;

//Configuração do carregamento de dados         
IDataView fullData = mlContext.Data.LoadFromTextFile<ModelInput>(path: housingPath, hasHeader: true, 
                                                                         separatorChar: ',', allowQuoting: true, allowSparse: false);

// Divide o conjunto de dados em duas partes: TrainingDataset (80%) e TestDataset (20%)
var trainTestData = mlContext.Data.TrainTestSplit(fullData, testFraction: 0.2);
trainDataView = trainTestData.TrainSet;
testDataView = trainTestData.TestSet;

// STEP 2: Run AutoML experiment
Console.WriteLine($"Running AutoML regression experiment for {ExperimentTime} seconds...");

var settings = new RegressionExperimentSettings
           {
               MaxExperimentTimeInSeconds = ExperimentTime,
               OptimizingMetric = RegressionMetric.MeanAbsoluteError
           };

var labelColumnInfo = new ColumnInformation()
{
    LabelColumnName = "Label"
};

var progress = new Progress<RunDetail<RegressionMetrics>>(p => 
{
    if (p.ValidationMetrics != null)
    {    
        Console.WriteLine($"Current Result – TrainerName: {p.TrainerName},  Runtime In Seconds: {p.RuntimeInSeconds:0.##}, RSquared: {p.ValidationMetrics.RSquared:0.##}, MeanAbsoluteError: {p.ValidationMetrics.MeanAbsoluteError:0.##}");
        //Console.WriteLine($"Current Result – TrainerName: {p.TrainerName},  Runtime In Seconds: {p.RuntimeInSeconds}, RSquared: {p.ValidationMetrics.RSquared}, MeanAbsoluteError: {p.ValidationMetrics.MeanAbsoluteError}");
    }
});


var experiment = mlContext.Auto().CreateRegressionExperiment(settings);

var experimentResult = experiment.Execute(trainDataView, labelColumnInfo, progressHandler: progress);

Running AutoML regression experiment for 10 seconds...
Current Result – TrainerName: SdcaRegression,  Runtime In Seconds: 0,17, RSquared: 0,63, MeanAbsoluteError: 51479,47
Current Result – TrainerName: SdcaRegression,  Runtime In Seconds: 0,1665812, RSquared: 0,6272549238572442, MeanAbsoluteError: 51479,46578746178
Current Result – TrainerName: FastTreeRegression,  Runtime In Seconds: 0,29, RSquared: 0,83, MeanAbsoluteError: 31970,34
Current Result – TrainerName: FastTreeRegression,  Runtime In Seconds: 0,2891977, RSquared: 0,8276824825461914, MeanAbsoluteError: 31970,344956517583
Current Result – TrainerName: FastTreeTweedieRegression,  Runtime In Seconds: 0,34, RSquared: 0,82, MeanAbsoluteError: 32635,12
Current Result – TrainerName: FastTreeTweedieRegression,  Runtime In Seconds: 0,3369162, RSquared: 0,8216709829703559, MeanAbsoluteError: 32635,116353688838
Current Result – TrainerName: FastForestRegression,  Runtime In Seconds: 0,42, RSquared: 0,66, MeanAbsoluteError: 49037,01
Curr

In [32]:
// STEP 3: Print metric from best model
var bestRun = experimentResult.BestRun;
Console.WriteLine($"=====================================================");
Console.WriteLine($"Total models produced: {experimentResult.RunDetails.Count()} --> Best model's trainer: {bestRun.TrainerName}");
Console.WriteLine($"Métricas --");
PrintMetrics(bestRun.ValidationMetrics);

Total models produced: 18 --> Best model's trainer: FastTreeRegression
Métricas --
MeanAbsoluteError: 31970,34
MeanSquaredError: 2264295999,14
RootMeanSquaredError: 47584,62
RSquared: 0,83


In [39]:
// STEP 5: Evaluate test data
IDataView testDataViewWithBestScore = bestRun.Model.Transform(testDataView);
RegressionMetrics testMetrics = mlContext.Regression.Evaluate(testDataViewWithBestScore, labelColumnName: "Label");

Console.WriteLine($"Metrics of best model on test data --");
//Console.WriteLine(Environment.NewLine);
PrintMetrics(testMetrics);

Metrics of best model on test data --
MeanAbsoluteError: 32457,33
MeanSquaredError: 2377334729,89
RootMeanSquaredError: 48757,92
RSquared: 0,82


In [13]:
// STEP 6: Save the best model for later deployment and inferencing
using (FileStream fs = File.Create(ModelPath))
    mlContext.Model.Save(bestRun.Model, trainDataView.Schema, fs);

In [24]:
// STEP 7: Create prediction engine from the best trained model
var predictionEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(bestRun.Model);

// STEP 8: Initialize a new test taxi trip, and get the predicted fare
var sampleData = new ModelInput()
{
    Longitude = -122.23F,
    Latitude = 37.88F,
    Housing_median_age = 41F,
    Total_rooms = 880F,
    Total_bedrooms = 129F,
    Population = 322F,
    Households = 126F,
    Median_income = 8.3252F,
    Ocean_proximity = @"NEAR BAY",
};

var resultprediction = predictionEngine.Predict (sampleData);

Console.WriteLine("Utilizando os dados abaixo: ");
Console.WriteLine(Environment.NewLine);
Console.WriteLine($"Longitude: {sampleData.Longitude}");
Console.WriteLine($"Latitude: {sampleData.Latitude}");
Console.WriteLine($"Housing_median_age: {sampleData.Housing_median_age}");
Console.WriteLine($"Total_rooms: {sampleData.Total_rooms}");
Console.WriteLine($"Total_bedrooms: {sampleData.Total_bedrooms}");
Console.WriteLine($"Population: {sampleData.Population}");
Console.WriteLine($"Households: {sampleData.Households}");
Console.WriteLine($"Median_income: {sampleData.Median_income}");
Console.WriteLine($"Ocean_proximity: {sampleData.Ocean_proximity}");
Console.WriteLine(Environment.NewLine);
Console.WriteLine ($"O Preço previsto para este imóvel é: {resultprediction.Score}");

Utilizando os dados abaixo: 


Longitude: -122,23
Latitude: 37,88
Housing_median_age: 41
Total_rooms: 880
Total_bedrooms: 129
Population: 322
Households: 126
Median_income: 8,3252
Ocean_proximity: NEAR BAY


O Preço previsto para este imóvel é: 462156,66
